# This notebook creates the Unseen Target Word test dataset using a subsection of the Kurrek et al. (2020) Slur Corpus.

In [ ]:
import pandas as pd

Reading the data and examining the distribution of slurs and gold labels.

In [ ]:
dataset= pd.read_csv('Notebook_6_14_kurrek.2020.slur-corpus.csv')

In [ ]:
dataset['slur'].value_counts()

tranny    13334
nigger    13334
faggot    13332
Name: slur, dtype: int64

In [ ]:
unseen_word = dataset[dataset['slur'] == 'tranny']

In [ ]:
unseen_word['gold_label'].value_counts(normalize = True)*100

DEG    57.312134
NDG    25.251237
HOM    14.369282
APR     2.467377
CMP     0.599970
Name: gold_label, dtype: float64

In [ ]:
unseen_word = unseen_word.copy()  # Create a copy of the DataFrame
unseen_word['hate_label'] = unseen_word['gold_label'].apply(lambda x: 1 if x == 'DEG' else 0)

In [ ]:
unseen_word = unseen_word.rename(columns={'body': 'sentences'})

In [ ]:
unseen_word['dataset'] = 'kurrek_unseen'

Creating columns for the target words as was done in Notebook 6 for the fine-tuning datset.

In [ ]:
# Import necessary libraries
import pandas as pd
import string

# Define the target_word function to identify target keywords in a sentence
def target_word(sentence):
    # List of primary keywords to search for
    keywords = ['bitch', 'drag', 'fag', 'gay', 'homo', 'puss', 'queen', 'queer', 'sissy','slay', 'slut', 'whore']

    # Variations of some keywords
    variations = {
        'whore': ["hoe", "ho", "ho's"], # Variations of 'whore'
    }

    # Use a set to store unique keywords found in the sentence
    found_keywords = set()

    # Split the sentence into words
    words = sentence.split()

    # Remove punctuation from words and make them lowercase for uniform processing
    cleaned_words = [word.strip(string.punctuation).lower() for word in words]

    # Check each keyword against the cleaned words in the sentence
    for keyword in keywords:
        if keyword == 'whore':
            # For the keyword 'whore', also check its variations
            if keyword in cleaned_words or any(var in cleaned_words for var in variations.get(keyword, [])):
                found_keywords.add(keyword)
        else:
            if any(keyword in word for word in cleaned_words):
                found_keywords.add(keyword)

    # Return found keywords or 'other' if no keywords are found
    return found_keywords if found_keywords else {'other'}

# List of keywords including 'other'
keywords = ['bitch', 'drag', 'fag', 'gay', 'homo', 'other', 'puss', 'queen', 'queer', 'sissy','slay', 'slut', 'whore']

# Assuming unseen_word is a DataFrame, apply the target_word function to create the 'key_words' column
unseen_word['key_words'] = unseen_word['sentences'].apply(target_word)

# For each keyword, create a new column in the DataFrame and populate with binary values (1 for present, 0 for absent)
for keyword in keywords:
    unseen_word[keyword] = unseen_word['key_words'].apply(lambda keywords: 1 if keyword in keywords else 0)

# Drop the 'key_words' column from the DataFrame as it's no longer needed
unseen_word.drop('key_words', axis=1, inplace=True)


Dropping other unecessary columsn from the dataframe

In [ ]:
unseen_word.drop(columns = ['id', 'link_id', 'parent_id', 'score', 'subreddit', 'author', 'slur', 'disagreement' ], inplace = True)

In [ ]:
unseen_word

,sentences,gold_label,hate_label,dataset,bitch,drag,fag,gay,homo,other,puss,queen,queer,sissy,slay,slut,whore
0,Fuck that I dont wanna watch tranny porn.,DEG,1,kurrek_unseen,0,0,0,0,0,1,0,0,0,0,0,0,0
1,Opie just wanted to have a good time. The same...,DEG,1,kurrek_unseen,0,0,0,0,0,0,0,1,0,0,0,0,0
4,Who even uses the word tranny except for trans...,NDG,0,kurrek_unseen,0,0,0,0,0,1,0,0,0,0,0,0,0
10,"I'd say the red pilling happened anyway, since...",DEG,1,kurrek_unseen,0,0,0,0,0,1,0,0,0,0,0,0,0
18,"Yeah, it's a really noticeable trend that none...",DEG,1,kurrek_unseen,0,0,0,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39977,There aren't enough trans people to have an im...,NDG,0,kurrek_unseen,0,0,0,0,0,1,0,0,0,0,0,0,0
39979,Yea but the radiator looks to have been ripped...,HOM,0,kurrek_unseen,0,0,0,0,0,1,0,0,0,0,0,0,0
39983,Atlanta is gold. One of my favorite [reaction ...,DEG,1,kurrek_unseen,0,0,0,0,0,1,0,0,0,0,0,0,0
39985,It should be required reading for every single...,NDG,0,kurrek_unseen,0,0,0,0,0,1,0,0,0,0,0,0,0


Downloading the dataframe as a CSV

In [ ]:
from google.colab import files
unseen_word.to_csv('unseen_word_testset.csv', index = False)
files.download('unseen_word_testset.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>